In [ ]:
from label_studio_sdk import Client
from ultralytics import YOLO
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
from pathlib import Path
import os
import glob
import shutil

In [ ]:
LABEL_STUDIO_URL = "http://localhost:8080"
API_KEY = ""
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
ls.check_connection()

In [ ]:
model = YOLO("runs/detect/OpenSource_Full_V1/weights/best.pt")

In [ ]:
val_study = ls.get_project(2)

In [ ]:
tasks = val_study.get_tasks()

In [ ]:
new_tasks = []

for _, row in tqdm(
    sampled_df.iterrows(),
    total=sampled_df.shape[0],
    desc="Copying and annotating images",
):
    image_path = row["image_path"]

    if os.path.exists(image_path):  # Ensure the source image exists
        # Extract the image file name from the path
        image_name = os.path.basename(image_path)

        # Create the destination path
        dest_path_jpg = os.path.join(dest_folder, image_name)

        if not os.path.exists(dest_path_jpg):
            # Copy the image to the destination
            shutil.copy(image_path, dest_path_jpg)
            img_url = f"/data/local-files/?d=GitHub/SquareEyes/{dest_path_jpg}"
            pred = model.predict(dest_path_jpg, verbose=False)
            new_preds = []
            for box in pred[0].boxes:
                label = int(box.cls[0])
                confidence = float(box.conf[0])
                x_c, y_c, w, h = box.xywhn[0]
                x = float(x_c - w / 2) * 100
                y = float(y_c - h / 2) * 100
                w, h = float(w) * 100, float(h) * 100
                org_h, org_w = box.orig_shape[0], box.orig_shape[0]
                new_preds.append(
                    {
                        "type": "rectanglelabels",
                        "value": {
                            "x": x,
                            "y": y,
                            "width": w,
                            "height": h,
                            "rectanglelabels": [pred[0].names[label]],
                            "score": confidence,
                        },
                        "to_name": "image",
                        "from_name": "label",
                        "image_rotation": 0,
                        "original_width": org_w,
                        "original_height": org_h,
                    },
                )
            new_tasks.append(
                {
                    "data": {"image": img_url},
                    "predictions": {
                        "model_version": "Full_Base_V1",
                        "result": new_preds,
                    },
                }
            )

    else:
        print(f"Image not found: {image_path}")

In [ ]:
new_tasks = [
    {"data": item["data"], "predictions": [item["predictions"]]} for item in new_tasks
]
val_study.import_tasks(new_tasks)

In [2]:
(743+199+83)/2757

0.3717809212912586